In [11]:
install.packages("BiocManager")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [ ]:
BiocManager::install('SingleCellExperiment')
BiocManager::install('BiocGenerics')
BiocManager::install(c('scater', 'scran', 'uwot'))
BiocManager::install(c('Rtsne'))

In [2]:
library(BiocGenerics)

In [3]:
library(parallel)

In [5]:
library(GenomicRanges)

In [6]:
library(stats4)

In [7]:
library(SummarizedExperiment)

In [10]:
library(SingleCellExperiment)

In [11]:
library(scater)

In [12]:
library(scran)

In [13]:
library(uwot)

Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:S4Vectors’:

    expand



In [14]:
library(Rtsne)

# 1) The SingleCellExperiment class
This class contains different slots. Those slots are called:
- **assays** (count matrix)
- **colData** (cells)
- **rowData** (genes)
- **sizeFactors** (normalization factors)
- **reduceDims** (results of PCA etc.)
- **metadata** (other results)  
- **spikeNames** (ERCC)

The **assays** and **reduceDim** slots can contain several layers. That means they can contain data of the same dimension but with other values.
<p align="center">
 <img src="https://osca.bioconductor.org/images/SingleCellExperiment.png" width="600">
</p>  


# 2) Create an SCE object
### Create count matrix
First we create a data.frame and specify column and row names.  
We then transform it into a matrix, which is the required data type for the core of the SCE object.  


In [24]:
counts_matrix <- data.frame(cell_1 = rpois(10, 10), 
                            cell_2 = rpois(10, 10), 
                            cell_3 = rpois(10, 30))
rownames(counts_matrix) <- paste0("gene_", 1:10)
counts_matrix <- as.matrix(counts_matrix) # must be a matrix object!

### Instatiate the SCE object
We could have added more contens in the list command like so  
`list(counts = count_matrix, log_counts = log_count_matrix)`  
in order to attach multiple levels to the assays slot of our SCE object.

In [ ]:
sce <- SingleCellExperiment(assays = list(counts = counts_matrix))

Some list exercicse to understand how we created the SCE object.  

In [3]:
lis = list(counts = counts_matrix)
lis$counts[1, 1]

[1] 9

# 3) SCE slots
## Count matrix (assays)
In the slot called "assays" one can save different kinds of count matrices. You can save one that cotains the raw data, another one that is log transformed etc.

In [37]:
# Look at all existing assays
assays(sce)
# Access the assay called "counts"
assay(sce, "counts")
# Access specific values of count matrix
assay(sce, "counts")[2,3]


List of length 2
names(2): counts logcounts

,cell_1,cell_2,cell_3
gene_1,7,11,32
gene_2,15,9,26
gene_3,14,5,27
gene_4,12,9,23
gene_5,6,9,33
gene_6,16,12,24
gene_7,13,7,27
gene_8,10,9,29
gene_9,12,7,34
gene_10,14,14,28


[1] 26

Let's add another layer to our assays slot and then trye to access it. We do this with a built in function that normalized and automatically adds the normalized count matrix as another layer.

In [34]:
# Save our old SCE object in a new one and add another layer to the slot assays
sce_new = sce
sce_new = scran::computeSumFactors(sce_new)
sce_new = scater::normalize(sce_new)
# Look at all existing assays
assays(sce_new)

Warning message in .local(object, ...):
“using library sizes as size factors”Warning message:
“'centreSizeFactors' is deprecated.
See help("Deprecated")”

List of length 1
names(1): counts

We can also add another layer to assays manually. To demonstrate that we add 100 to every entry in the layer "counts" of the assays slot in sce and assign it to counts_100. We then add this layer manually to our new SCE object called sce_new_2.

In [36]:
sce_new_2 = sce
counts_100 <- assay(sce, "counts") + 100
assay(sce_new_2, "counts_100") <- counts_100
assays(sce_new_2)

List of length 3
names(3): counts logcounts counts_100

## Cells (colData)

We can add information about the cells such as batch number or cell id.

In [45]:
# Create a data.frame object with two variable that we can attach to our SCE object
cell_metadata <- data.frame(batch = c(1, 1, 2))
rownames(cell_metadata) <- paste0("cell_", 1:3)
cell_metadata$"cell_id" = c(23, 45, 21)

# Add the data.frame object to our colData slot
colData(sce_new_2) = DataFrame(cell_metadata)
colData(sce_new_2)

DataFrame with 3 rows and 2 columns
           batch   cell_id
       <numeric> <numeric>
cell_1         1        23
cell_2         1        45
cell_3         2        21

In [47]:
# Access elements in colData
colData(sce_new_2)[1,2]

[1] 23

## Genes (rowData)
We can also add annotations to our genes. The simplest way to do that is to just calculate some qc metric as is shown below. But you can also add columns to this slot manually.

In [54]:
sce_new_2 <- scater::calculateQCMetrics(sce_new_2)
colnames(rowData(sce_new_2))
rowData(sce_new_2)[,c("mean_counts","n_cells_by_counts")]

Warning message:
“'calculateQCMetrics' is deprecated.
Use 'perCellQCMetrics' or 'perFeatureQCMetrics' instead.”

[1] "is_feature_control"    "mean_counts"           "log10_mean_counts"    
[4] "n_cells_by_counts"     "pct_dropout_by_counts" "total_counts"         
[7] "log10_total_counts"

DataFrame with 10 rows and 2 columns
             mean_counts n_cells_by_counts
               <numeric>         <integer>
gene_1  16.6666666666667                 3
gene_2  16.6666666666667                 3
gene_3  15.3333333333333                 3
gene_4  14.6666666666667                 3
gene_5                16                 3
gene_6  17.3333333333333                 3
gene_7  15.6666666666667                 3
gene_8                16                 3
gene_9  17.6666666666667                 3
gene_10 18.6666666666667                 3

## (EXTRA) Subsetting using colData and rowData
We can use both colData and rowData to subset our count matrix.

In [57]:
# Subset count matrix data with row Data
assay(sce_new_2, 'counts_100')[, colData(sce_new_2)$cell_id < 40]

# Subset count matrix data with column Data
assay(sce_new_2, 'counts_100')[rowData(sce_new_2)$mean_counts < 16,]

,cell_1,cell_3
gene_1,107,132
gene_2,115,126
gene_3,114,127
gene_4,112,123
gene_5,106,133
gene_6,116,124
gene_7,113,127
gene_8,110,129
gene_9,112,134
gene_10,114,128


,cell_1,cell_2,cell_3
gene_3,114,105,127
gene_4,112,109,123
gene_7,113,107,127


## SizeFactors
This is another slot, next to assays, rowData and colData. This slot is calculated when calling the functions below (as has been done at the very beginning). The data.frame in this slot ocntains information on normalization factors that was used to produce normalized data.

In [ ]:
sce_new_2 <- scran::computeSumFactors(sce)
sce_new_2 <- scater::normalize(sce)

In [59]:
sizeFactors(sce_new_2)

cell_1    cell_2    cell_3 
0.7226721 0.5587045 1.7186235

## reduceDims
This is another slot that is very useful to store results of PCA, tSNE, UMAP etc.

In [81]:
# do PCA
sce <- scater::runPCA(sce)
reducedDim(sce, "PCA")

# do tSNE
sce <- scater::runTSNE(sce, perplexity = 0.1)
reducedDim(sce, "TSNE")

# show all reduced dimenisons of our count matrix
reducedDims(sce)

Warning message in check_numbers(k = k, nu = nu, nv = nv, limit = min(dim(x)) - :
“more singular values/vectors requested than available”Warning message in (function (A, nv = 5, nu = nv, maxit = 1000, work = nv + 7, reorth = TRUE, :
“You're computing too large a percentage of total singular values, use a standard svd instead.”

,PC1,PC2
cell_1,-1.0982660,-0.1074447
cell_2,0.6811321,-0.6167531
cell_3,0.4171339,0.7241978


Perplexity should be lower than K!


cell_1,-1639.265,5454.288
cell_2,-3889.309,-4179.526
cell_3,5528.575,-1274.762


List of length 2
names(2): PCA TSNE

We may want to add a new layer in the slot "reduceDims" manually. This makes sense when we want to run a dimensionlity reduction algorithm that is not yet implemented directly with the SCE class. This is possible and straightforward as shown below.  
Below the function `umap()` from the package `uwot` is run directly as opposed to through the scater package (which would be `scater::runUMAP()`).

In [83]:
u <- uwot::umap(t(logcounts(sce)), n_neighbors = 2)
reducedDim(sce, "UMAP_uwot") <- u

reducedDim(sce, "UMAP_uwot")
reducedDims(sce)

cell_1,-0.6426075,-0.1031802
cell_2,0.4711351,0.4533947
cell_3,0.1714724,-0.3502145


List of length 3
names(3): PCA TSNE UMAP_uwot

## metadata
During the analysis of scRNA-seq data, we may produce some data that does not fit into any other slot conceptually. But there is another slot called "metadata" where we can put this kind of data to.  
You can put anything there. This slot is basically a list, where each list entry corresponds to an object. This object can be a vector, a table or a whole big data frame or matrix.

In [109]:
my_genes <- c("gene_1", "gene_5")
metadata(sce)$mygenes = c("gene_1", "gene_5")
metadata(sce)$his_genes = list(c("ggg", "hh"), c("jj", "oo"))

# create some data.frame
a_frame = data.frame(name=c("hans", "robert", "ludwig"), age=c(23, 56, 12), birth_place=c("lauf", "stadeln", "obetrtrubach"))
rownames(a_frame) = a_frame[,"name"]
a_frame = a_frame[,-1]

# attach the data.frame to metadata
metadata(sce)$my_list = a_frame

metadata(sce)

,age,birth_place
,<dbl>,<fct>
hans,23,lauf
robert,56,stadeln
ludwig,12,obetrtrubach


## spikeNames
An interesitng tutorial for that can be found [here](https://bioconductor.org/packages/release/bioc/vignettes/SingleCellExperiment/inst/doc/intro.html)